Copyright 2021 Floq authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [ ]:
#@title Copyright 2021 Floq authors, All Rights Reserved.
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In this tutorial, we introduce how to use TensorFlow Quantum with Floq backend. 

# Setup

Firstly, Please make sure that the versions of all libraries should match for the successful execution.

In [ ]:
!pip install tensorflow==2.4.1 --quiet
!pip install tensorflow-quantum==0.5.0 --quiet
!pip install floq-client==0.1.1 --quiet

     |████████████████████████████████| 394.3MB 42kB/s 
     |████████████████████████████████| 471kB 37.3MB/s 
     |████████████████████████████████| 2.9MB 8.7MB/s 
     |████████████████████████████████| 3.8MB 20.8MB/s 
     |████████████████████████████████| 7.8MB 3.3MB/s 
     |████████████████████████████████| 92kB 11.1MB/s 
     |████████████████████████████████| 92kB 10.9MB/s 
     |████████████████████████████████| 3.0MB 30.3MB/s 
     |████████████████████████████████| 5.6MB 17.7MB/s 
     |████████████████████████████████| 1.3MB 35.2MB/s 
     |████████████████████████████████| 102kB 11.8MB/s 
     |████████████████████████████████| 389kB 57.2MB/s 
     |████████████████████████████████| 1.5MB 36.5MB/s 
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.30.0 which is incompatible.
     |████████████████████████████████| 51kB 2.1MB/s 
     |████████████████████████████████| 51kB 3.7MB/s 
     |████████████████████████████████| 1.8MB 5.0MB/s 
    

In [ ]:
import floq.client
import tensorflow_quantum as tfq
import cirq
import numpy as np
import sys

num_qubits = 26  # The minimum number of qubits is 26 for Floq.
np.random.seed(0)

# Circuit preparation

## Construct a Cirq circuit

In [ ]:
qubits = cirq.GridQubit.rect(1, num_qubits)  # TFQ only accepts cirq.GridQubit
my_circuit = cirq.Circuit(cirq.H.on_each(qubits))
for q1, q2 in zip(qubits[-2::-1], qubits[::-1]):
  my_circuit += cirq.Circuit(cirq.CNOT.on(q1, q2))
my_circuit

(0, 0): ────H───────────────────────────────────────────────────────────────────────────────────────────────────@───
                                                                                                                │
(0, 1): ────H───────────────────────────────────────────────────────────────────────────────────────────────@───X───
                                                                                                            │
(0, 2): ────H───────────────────────────────────────────────────────────────────────────────────────────@───X───────
                                                                                                        │
(0, 3): ────H───────────────────────────────────────────────────────────────────────────────────────@───X───────────
                                                                                                    │
(0, 4): ────H───────────────────────────────────────────────────────────────────────────────────@───X───────────────
                                                                                                │
(0, 5): ────H───────────────────────────────────────────────────────────────────────────────@───X───────────────────
                                                                                            │
(0, 6): ────H───────────────────────────────────────────────────────────────────────────@───X───────────────────────
                                                                                        │
(0, 7): ────H───────────────────────────────────────────────────────────────────────@───X───────────────────────────
                                                                                    │
(0, 8): ────H───────────────────────────────────────────────────────────────────@───X───────────────────────────────
                                                                                │
(0, 9): ────H───────────────────────────────────────────────────────────────@───X───────────────────────────────────
                                                                            │
(0, 10): ───H───────────────────────────────────────────────────────────@───X───────────────────────────────────────
                                                                        │
(0, 11): ───H───────────────────────────────────────────────────────@───X───────────────────────────────────────────
                                                                    │
(0, 12): ───H───────────────────────────────────────────────────@───X───────────────────────────────────────────────
                                                                │
(0, 13): ───H───────────────────────────────────────────────@───X───────────────────────────────────────────────────
                                                            │
(0, 14): ───H───────────────────────────────────────────@───X───────────────────────────────────────────────────────
                                                        │
(0, 15): ───H───────────────────────────────────────@───X───────────────────────────────────────────────────────────
                                                    │
(0, 16): ───H───────────────────────────────────@───X───────────────────────────────────────────────────────────────
                                                │
(0, 17): ───H───────────────────────────────@───X───────────────────────────────────────────────────────────────────
                                            │
(0, 18): ───H───────────────────────────@───X───────────────────────────────────────────────────────────────────────
                                        │
(0, 19): ───H───────────────────────@───X───────────────────────────────────────────────────────────────────────────
                                    │
(0, 20): ───H───────────────────@───X───────────────────────────────────────────────────────────────────────────────
                       

## Define Observables

In [ ]:
observables = [cirq.Y(qubits[0]), 2.0 * cirq.Z(qubits[0]) - 1.0 * cirq.X(qubits[1])]

## Convert them into TF tensor for TFQuantum

In [ ]:
my_circuit_tensor = tfq.convert_to_tensor([my_circuit])
observables_tensor = tfq.convert_to_tensor([observables])

# Simulation with Floq

## Create a Floq Client & Simulator with your API key

In [ ]:
API_KEY = "YOUR_API_KEY" #@param {type:"string"}
client = floq.client.CirqClient(API_KEY)
sim = client.simulator

## Construct a TFQ Expectation layer

In [ ]:
tfq_floq_exp_layer = tfq.layers.Expectation(backend=sim)
exp = tfq_floq_exp_layer(my_circuit_tensor, operators=observables_tensor)

In [ ]:
exp

## Construct a TFQ sample layer

In [ ]:
tfq_floq_sample_layer = tfq.layers.Sample(backend=sim)
sample_ragged_tensor = tfq_floq_sample_layer(my_circuit_tensor, repetitions=10000)

In [ ]:
sample_ragged_tensor

In [ ]:
sample_tensor = sample_ragged_tensor.values.to_tensor()

In [ ]:
sample_tensor